# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will:
* Remove duplicates
* Remove NaN entires


In [1]:
import pandas as pd
import re
from string import punctuation
import numpy as np
import emoji
import sys
sys.path.append("../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
priority = pd.read_csv("full_dataset_priority_high_med_low.csv" , index_col=0)
priority

,id,type,created_at,repo,repo_url,action,title,labels,body,index,class
0,1.675131e+10,IssuesEvent,2021-06-12 00:20:05,NoahOrtega/Blue-Coin-Tracker,https://api.github.com/repos/NoahOrtega/Blue-C...,closed,Setting darkmode during launch causes crash on...,bug high priority,Setting dark mode in onCreate() in the main ac...,1.0,high_priority
1,1.675179e+10,IssuesEvent,2021-06-12 02:13:44,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Context Prioritization Ignore not working,High Priority bug,**Describe the bug**\r\nWhen I add a folder pa...,1.0,high_priority
2,1.675179e+10,IssuesEvent,2021-06-12 02:14:42,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Search text glitches,High Priority UI/UX bug,**Describe the bug**\r\nAfter invoking the sea...,1.0,high_priority
3,1.675189e+10,IssuesEvent,2021-06-12 02:40:12,SwatInc/CD4,https://api.github.com/repos/SwatInc/CD4,closed,Generate report by Episode number,High Priority Reporting enhancement,"The current reports are all, generated by samp...",1.0,high_priority
4,1.675203e+10,IssuesEvent,2021-06-12 03:17:18,eclipse/che-che4z-explorer-for-endevor,https://api.github.com/repos/eclipse/che-che4z...,opened,Error files.encoding generate ï¿½ or � and tha...,High Priority,We have an issue when we browse or retrieve an...,1.0,high_priority
...,...,...,...,...,...,...,...,...,...,...,...
26371,1.593657e+10,IssuesEvent,2021-04-14 11:20:08,inf112-v21/Trivial-Solution,https://api.github.com/repos/inf112-v21/Trivia...,closed,Som spiller vil jeg ha flere forskjellige kart...,Low priority User Story,Krav:\r\n- Antall maps i mappen 'maps' skal væ...,1.0,low_priority
26372,1.593706e+10,IssuesEvent,2021-04-14 11:59:33,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Catch general exception,good practices low priority repair,* some routes do throw exceptions other than t...,1.0,low_priority
26373,1.593712e+10,IssuesEvent,2021-04-14 12:03:28,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Backend reset password route,enhancement low priority,* Add `POST /auth/reset_request` route which r...,1.0,low_priority
26374,1.593712e+10,IssuesEvent,2021-04-14 12:04:08,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Frontend reset password,enhancement low priority,* Add `Forgot Password` button on login page\r...,1.0,low_priority


In [3]:
#Number of issues per class
priority["class"].value_counts()

class
high_priority      14334
low_priority        7379
medium_priority     4659
Name: count, dtype: int64

In [4]:
# Split the dataframe 
high_priority = priority.loc[priority['class'] == 'high_priority'].copy()
medium_priority = priority.loc[priority['class'] == 'medium_priority'].copy()
low_priority = priority.loc[priority['class'] == 'low_priority'].copy()

In [5]:
high_priority.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14334 entries, 0 to 14337
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          14334 non-null  float64
 1   type        14334 non-null  object 
 2   created_at  14334 non-null  object 
 3   repo        14334 non-null  object 
 4   repo_url    14334 non-null  object 
 5   action      14334 non-null  object 
 6   title       14334 non-null  object 
 7   labels      14334 non-null  object 
 8   body        12376 non-null  object 
 9   index       14334 non-null  float64
 10  class       14334 non-null  object 
dtypes: float64(2), object(9)
memory usage: 1.3+ MB


In [6]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
high_priority["label"] = 0
high_priority.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,index,class,label
0,1.675131e+10,IssuesEvent,2021-06-12 00:20:05,NoahOrtega/Blue-Coin-Tracker,https://api.github.com/repos/NoahOrtega/Blue-C...,closed,Setting darkmode during launch causes crash on...,bug high priority,Setting dark mode in onCreate() in the main ac...,1.0,high_priority,0
1,1.675179e+10,IssuesEvent,2021-06-12 02:13:44,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Context Prioritization Ignore not working,High Priority bug,**Describe the bug**\r\nWhen I add a folder pa...,1.0,high_priority,0
2,1.675179e+10,IssuesEvent,2021-06-12 02:14:42,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Search text glitches,High Priority UI/UX bug,**Describe the bug**\r\nAfter invoking the sea...,1.0,high_priority,0
3,1.675189e+10,IssuesEvent,2021-06-12 02:40:12,SwatInc/CD4,https://api.github.com/repos/SwatInc/CD4,closed,Generate report by Episode number,High Priority Reporting enhancement,"The current reports are all, generated by samp...",1.0,high_priority,0
4,1.675203e+10,IssuesEvent,2021-06-12 03:17:18,eclipse/che-che4z-explorer-for-endevor,https://api.github.com/repos/eclipse/che-che4z...,opened,Error files.encoding generate ï¿½ or � and tha...,High Priority,We have an issue when we browse or retrieve an...,1.0,high_priority,0


In [7]:
medium_priority["label"] = 1
medium_priority.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,index,class,label
14338,1.675181e+10,IssuesEvent,2021-06-12 02:18:55,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Drag and drop often shows restricted icon (🚫),Medium Priority bug,**Describe the bug**\r\nWhile trying to drag a...,1.0,medium_priority,1
14339,1.675184e+10,IssuesEvent,2021-06-12 02:27:07,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Ignore Tags,Medium Priority bug,There is a lot of tags which I don't know what...,1.0,medium_priority,1
14340,1.612969e+10,IssuesEvent,2021-04-29 01:16:17,StatisticsNZ/simplevis,https://api.github.com/repos/StatisticsNZ/simp...,opened,numeric breaks: log or log10 trans does not wo...,medium priority,NaN,1.0,medium_priority,1
14341,1.612976e+10,IssuesEvent,2021-04-29 01:27:17,KingSupernova31/RulesGuru,https://api.github.com/repos/KingSupernova31/R...,opened,"Add a ""notes"" field on questions",enhancement medium priority,"Questions should have a ""notes"" field. This fi...",1.0,medium_priority,1
14342,1.612978e+10,IssuesEvent,2021-04-29 01:31:03,KingSupernova31/RulesGuru,https://api.github.com/repos/KingSupernova31/R...,opened,Don't allow a question to have the same editor...,enhancement medium priority,"Right now, if an admin has both the ""Editor"" a...",1.0,medium_priority,1


In [8]:
low_priority["label"] = 2
low_priority.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,index,class,label
18997,1.675160e+10,IssuesEvent,2021-06-12 01:26:20,Sequel-Ace/Sequel-Ace,https://api.github.com/repos/Sequel-Ace/Sequel...,closed,slow to connect via proxysql,Feature Request Low priority PR Welcome stale,\r\n- Sequel Ace Version (including build numb...,1.0,low_priority,2
18998,1.675165e+10,IssuesEvent,2021-06-12 01:37:40,DoobDev/Doob,https://api.github.com/repos/DoobDev/Doob,reopened,Redo the Help menu!,Low Priority feature_request,It's ugly right now.\r\n\r\n- [ ] Separate com...,1.0,low_priority,2
18999,1.612979e+10,IssuesEvent,2021-04-29 01:32:47,KingSupernova31/RulesGuru,https://api.github.com/repos/KingSupernova31/R...,opened,Have discord bot automatically assign roles,enhancement low priority,"RG has internal roles for all its admins, and ...",1.0,low_priority,2
19000,1.612980e+10,IssuesEvent,2021-04-29 01:34:02,KingSupernova31/RulesGuru,https://api.github.com/repos/KingSupernova31/R...,opened,Jumping to editor forgets question number,bug good first issue low priority,"Typing the sequence of characters ""admin"" on a...",1.0,low_priority,2
19001,1.613008e+10,IssuesEvent,2021-04-29 02:19:14,inaturalist/iNaturalistAndroid,https://api.github.com/repos/inaturalist/iNatu...,closed,Prevent crashes from encoding problems in the ...,Low Priority bug,https://console.firebase.google.com/u/2/projec...,1.0,low_priority,2


In [9]:
#Non_priority = Non_priority.sample(frac=0.15)

In [10]:
priority = pd.concat([high_priority, medium_priority, low_priority] , ignore_index = True)
priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,index,class,label
26367,1.593657e+10,IssuesEvent,2021-04-14 11:20:08,inf112-v21/Trivial-Solution,https://api.github.com/repos/inf112-v21/Trivia...,closed,Som spiller vil jeg ha flere forskjellige kart...,Low priority User Story,Krav:\r\n- Antall maps i mappen 'maps' skal væ...,1.0,low_priority,2
26368,1.593706e+10,IssuesEvent,2021-04-14 11:59:33,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Catch general exception,good practices low priority repair,* some routes do throw exceptions other than t...,1.0,low_priority,2
26369,1.593712e+10,IssuesEvent,2021-04-14 12:03:28,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Backend reset password route,enhancement low priority,* Add `POST /auth/reset_request` route which r...,1.0,low_priority,2
26370,1.593712e+10,IssuesEvent,2021-04-14 12:04:08,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Frontend reset password,enhancement low priority,* Add `Forgot Password` button on login page\r...,1.0,low_priority,2
26371,1.593760e+10,IssuesEvent,2021-04-14 12:40:00,brian-team/brian2cuda,https://api.github.com/repos/brian-team/brian2...,closed,Sort EventMonitor data by index within each ti...,low priority usability,Currently our `EventMonitor` (~ `SpikeMonitor`...,1.0,low_priority,2


In [11]:
#data[data["repo"] == "microsoft/vscode"]

In [12]:
# Drop duplicates by the content of the body
all_priority = priority.drop_duplicates(subset=['body'], keep='last')

In [13]:
all_priority.dropna(inplace=True)

/tmp/ipykernel_88890/2768040423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority.dropna(inplace=True)


In [14]:
all_priority

,id,type,created_at,repo,repo_url,action,title,labels,body,index,class,label
1,1.675179e+10,IssuesEvent,2021-06-12 02:13:44,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Context Prioritization Ignore not working,High Priority bug,**Describe the bug**\r\nWhen I add a folder pa...,1.0,high_priority,0
2,1.675179e+10,IssuesEvent,2021-06-12 02:14:42,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Search text glitches,High Priority UI/UX bug,**Describe the bug**\r\nAfter invoking the sea...,1.0,high_priority,0
3,1.675189e+10,IssuesEvent,2021-06-12 02:40:12,SwatInc/CD4,https://api.github.com/repos/SwatInc/CD4,closed,Generate report by Episode number,High Priority Reporting enhancement,"The current reports are all, generated by samp...",1.0,high_priority,0
5,1.675205e+10,IssuesEvent,2021-06-12 03:20:33,eclipse/che-che4z-explorer-for-endevor,https://api.github.com/repos/eclipse/che-che4z...,opened,Generate issue with the plugin Broadcom Explo...,High Priority,E4E is generating in place. If I have an eleme...,1.0,high_priority,0
6,1.612937e+10,IssuesEvent,2021-04-29 00:27:52,CredentialEngine/CredentialRegistry,https://api.github.com/repos/CredentialEngine/...,opened,Sandbox has been unresponsive a lot today,High Priority,@excelsior @edgarf \r\nThe sandbox server is n...,1.0,high_priority,0
...,...,...,...,...,...,...,...,...,...,...,...,...
26367,1.593657e+10,IssuesEvent,2021-04-14 11:20:08,inf112-v21/Trivial-Solution,https://api.github.com/repos/inf112-v21/Trivia...,closed,Som spiller vil jeg ha flere forskjellige kart...,Low priority User Story,Krav:\r\n- Antall maps i mappen 'maps' skal væ...,1.0,low_priority,2
26368,1.593706e+10,IssuesEvent,2021-04-14 11:59:33,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Catch general exception,good practices low priority repair,* some routes do throw exceptions other than t...,1.0,low_priority,2
26369,1.593712e+10,IssuesEvent,2021-04-14 12:03:28,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Backend reset password route,enhancement low priority,* Add `POST /auth/reset_request` route which r...,1.0,low_priority,2
26370,1.593712e+10,IssuesEvent,2021-04-14 12:04:08,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Frontend reset password,enhancement low priority,* Add `Forgot Password` button on login page\r...,1.0,low_priority,2


In [15]:
# Resets the index
all_priority.reset_index(inplace=True)
all_priority

,level_0,id,type,created_at,repo,repo_url,action,title,labels,body,index,class,label
0,1,1.675179e+10,IssuesEvent,2021-06-12 02:13:44,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Context Prioritization Ignore not working,High Priority bug,**Describe the bug**\r\nWhen I add a folder pa...,1.0,high_priority,0
1,2,1.675179e+10,IssuesEvent,2021-06-12 02:14:42,adirh3/Fluent-Search,https://api.github.com/repos/adirh3/Fluent-Search,closed,Search text glitches,High Priority UI/UX bug,**Describe the bug**\r\nAfter invoking the sea...,1.0,high_priority,0
2,3,1.675189e+10,IssuesEvent,2021-06-12 02:40:12,SwatInc/CD4,https://api.github.com/repos/SwatInc/CD4,closed,Generate report by Episode number,High Priority Reporting enhancement,"The current reports are all, generated by samp...",1.0,high_priority,0
3,5,1.675205e+10,IssuesEvent,2021-06-12 03:20:33,eclipse/che-che4z-explorer-for-endevor,https://api.github.com/repos/eclipse/che-che4z...,opened,Generate issue with the plugin Broadcom Explo...,High Priority,E4E is generating in place. If I have an eleme...,1.0,high_priority,0
4,6,1.612937e+10,IssuesEvent,2021-04-29 00:27:52,CredentialEngine/CredentialRegistry,https://api.github.com/repos/CredentialEngine/...,opened,Sandbox has been unresponsive a lot today,High Priority,@excelsior @edgarf \r\nThe sandbox server is n...,1.0,high_priority,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19606,26367,1.593657e+10,IssuesEvent,2021-04-14 11:20:08,inf112-v21/Trivial-Solution,https://api.github.com/repos/inf112-v21/Trivia...,closed,Som spiller vil jeg ha flere forskjellige kart...,Low priority User Story,Krav:\r\n- Antall maps i mappen 'maps' skal væ...,1.0,low_priority,2
19607,26368,1.593706e+10,IssuesEvent,2021-04-14 11:59:33,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Catch general exception,good practices low priority repair,* some routes do throw exceptions other than t...,1.0,low_priority,2
19608,26369,1.593712e+10,IssuesEvent,2021-04-14 12:03:28,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Backend reset password route,enhancement low priority,* Add `POST /auth/reset_request` route which r...,1.0,low_priority,2
19609,26370,1.593712e+10,IssuesEvent,2021-04-14 12:04:08,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Frontend reset password,enhancement low priority,* Add `Forgot Password` button on login page\r...,1.0,low_priority,2


In [16]:
# Does this need to be more balanced?
all_priority["label"].value_counts()

label
0    10497
2     5683
1     3431
Name: count, dtype: int64

In [17]:
all_priority.drop(columns=["index"] , inplace= True)

/tmp/ipykernel_88890/2269905686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority.drop(columns=["index"] , inplace= True)


In [18]:
# Copy content of body to a new col named text
# Why?
all_priority["text"] = all_priority["body"]
all_priority.tail()

/tmp/ipykernel_88890/1678713137.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority["text"] = all_priority["body"]


,level_0,id,type,created_at,repo,repo_url,action,title,labels,body,class,label,text
19606,26367,1.593657e+10,IssuesEvent,2021-04-14 11:20:08,inf112-v21/Trivial-Solution,https://api.github.com/repos/inf112-v21/Trivia...,closed,Som spiller vil jeg ha flere forskjellige kart...,Low priority User Story,Krav:\r\n- Antall maps i mappen 'maps' skal væ...,low_priority,2,Krav:\r\n- Antall maps i mappen 'maps' skal væ...
19607,26368,1.593706e+10,IssuesEvent,2021-04-14 11:59:33,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Catch general exception,good practices low priority repair,* some routes do throw exceptions other than t...,low_priority,2,* some routes do throw exceptions other than t...
19608,26369,1.593712e+10,IssuesEvent,2021-04-14 12:03:28,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Backend reset password route,enhancement low priority,* Add `POST /auth/reset_request` route which r...,low_priority,2,* Add `POST /auth/reset_request` route which r...
19609,26370,1.593712e+10,IssuesEvent,2021-04-14 12:04:08,Gird-the-Grid/Grid-the-Grid,https://api.github.com/repos/Gird-the-Grid/Gri...,opened,Frontend reset password,enhancement low priority,* Add `Forgot Password` button on login page\r...,low_priority,2,* Add `Forgot Password` button on login page\r...
19610,26371,1.593760e+10,IssuesEvent,2021-04-14 12:40:00,brian-team/brian2cuda,https://api.github.com/repos/brian-team/brian2...,closed,Sort EventMonitor data by index within each ti...,low priority usability,Currently our `EventMonitor` (~ `SpikeMonitor`...,low_priority,2,Currently our `EventMonitor` (~ `SpikeMonitor`...


In [19]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["text" , "label" , "class"]]

In [20]:
all_priority_subset

,text,label,class
0,**Describe the bug**\r\nWhen I add a folder pa...,0,high_priority
1,**Describe the bug**\r\nAfter invoking the sea...,0,high_priority
2,"The current reports are all, generated by samp...",0,high_priority
3,E4E is generating in place. If I have an eleme...,0,high_priority
4,@excelsior @edgarf \r\nThe sandbox server is n...,0,high_priority
...,...,...,...
19606,Krav:\r\n- Antall maps i mappen 'maps' skal væ...,2,low_priority
19607,* some routes do throw exceptions other than t...,2,low_priority
19608,* Add `POST /auth/reset_request` route which r...,2,low_priority
19609,* Add `Forgot Password` button on login page\r...,2,low_priority


In [21]:
# What is this for?
all_priority_subset.to_csv("subset_priority_dataset.csv")

In [22]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_88890/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [23]:
all_priority_subset

,text,label,class,text_str
0,**Describe the bug**\r\nWhen I add a folder pa...,0,high_priority,**Describe the bug**\r\nWhen I add a folder pa...
1,**Describe the bug**\r\nAfter invoking the sea...,0,high_priority,**Describe the bug**\r\nAfter invoking the sea...
2,"The current reports are all, generated by samp...",0,high_priority,"The current reports are all, generated by samp..."
3,E4E is generating in place. If I have an eleme...,0,high_priority,E4E is generating in place. If I have an eleme...
4,@excelsior @edgarf \r\nThe sandbox server is n...,0,high_priority,@excelsior @edgarf \r\nThe sandbox server is n...
...,...,...,...,...
19606,Krav:\r\n- Antall maps i mappen 'maps' skal væ...,2,low_priority,Krav:\r\n- Antall maps i mappen 'maps' skal væ...
19607,* some routes do throw exceptions other than t...,2,low_priority,* some routes do throw exceptions other than t...
19608,* Add `POST /auth/reset_request` route which r...,2,low_priority,* Add `POST /auth/reset_request` route which r...
19609,* Add `Forgot Password` button on login page\r...,2,low_priority,* Add `Forgot Password` button on login page\r...


In [25]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_88890/2393042031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [26]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "label"]]
priority_label_text

,text_clean,label
0,describe the bug when i add a folder path and ...,0
1,describe the bug after invoking the search bar...,0
2,the current reports are all generated by sampl...,0
3,e4e is generating in place if i have an elemen...,0
4,excelsior edgarf the sandbox server is not res...,0
...,...,...
19606,NaN,2
19607,some routes do throw exceptions other than the...,2
19608,add post authresetrequest route which receives...,2
19609,add forgot password button on login page add r...,2


In [27]:
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_88890/1038009599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_88890/1038009599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,label
0,describe the bug when i add a folder path and ...,0
1,describe the bug after invoking the search bar...,0
2,the current reports are all generated by sampl...,0
3,e4e is generating in place if i have an elemen...,0
4,excelsior edgarf the sandbox server is not res...,0
...,...,...
18453,device iphone 12 mini chromesafari case 2 in 4...,2
18454,some routes do throw exceptions other than the...,2
18455,add post authresetrequest route which receives...,2
18456,add forgot password button on login page add r...,2


In [28]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = medium priority, 2 low priority.
priority_label_text.to_csv("priority_high_med_low_dataset_clean.csv")

In [29]:
df2 = pd.read_csv("priority_high_med_low_dataset_clean.csv", index_col=0)
df2

,text_clean,label
0,describe the bug when i add a folder path and ...,0
1,describe the bug after invoking the search bar...,0
2,the current reports are all generated by sampl...,0
3,e4e is generating in place if i have an elemen...,0
4,excelsior edgarf the sandbox server is not res...,0
...,...,...
18453,device iphone 12 mini chromesafari case 2 in 4...,2
18454,some routes do throw exceptions other than the...,2
18455,add post authresetrequest route which receives...,2
18456,add forgot password button on login page add r...,2
